In [55]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [56]:
# 100,000 people with differing levels of covid symptoms
N_people = 100000
df = pd.DataFrame(np.arange(N_people), columns = ['person'])

# Potential outcomes (Y0): life-span if no vent
df['y0'] = np.random.normal(loc = 9.4, scale = 4, size = N_people)
df.loc[df['y0'] < 0, 'y0'] = 0

# Potential outcomes (Y1): life-span if assigned to vents
df['y1'] = np.random.normal(loc = 10, scale = 4, size = N_people)
df.loc[df['y1'] < 0, 'y1'] = 0

# Define individual treatment effect
df['delta'] = df['y1'] - df['y0']

In [57]:
df.describe()

,person,y0,y1,delta
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,49999.500000,9.412078,10.012359,0.600282
std,28867.657797,3.970169,3.974571,5.609362
min,0.000000,0.000000,0.000000,-21.708308
25%,24999.750000,6.695223,7.307506,-3.189576
50%,49999.500000,9.404218,10.005522,0.590902
75%,74999.250000,12.099146,12.688543,4.405935
max,99999.000000,25.662723,25.893349,23.137772


In [66]:
def do_calculations(df, assignment):
    if assignment == 'perfect':
        # Perfect doctor assigns vents (the treatment) only to those who benefit
        df['assignment'] = (df['delta'] > 0).astype('int32')
    elif assignment == 'bad':
        # or choose the bad doctor assignment half-half
        df['assignment'] = 0
        df.loc[0:len(df)//2, 'assignment'] = 1

    df['realized_outcome'] = (1 - df['assignment']) * df['y0'] + df['assignment'] * df['y1']

    # Calculate all aggregate Causal Parameters (ATE, ATT, ATU)
    ATE = np.mean(df['delta'])
    ATT = np.mean(df.loc[df['assignment'] == 1, 'delta'])
    ATU = np.mean(df.loc[df['assignment'] == 0, 'delta'])
    print('ATE:', ATE)
    print('ATT:', ATT)
    print('ATU:', ATU)

    # Use the switching equation to select realized outcomes from potential 
    # outcomes based on treatment assignment given by the Perfect Doctor
    po = {}
    po['E[y0 | D = 1]'] = np.mean(df.loc[df['assignment'] == 1, 'y0'])
    po['E[y0 | D = 0]'] = np.mean(df.loc[df['assignment'] == 0, 'y0'])
    print(po)

    sb = po['E[y0 | D = 1]'] - po['E[y0 | D = 0]'] 
    print('selection bias:', sb)

    pi = np.mean(df['assignment'])
    print('pi:', pi)

    HTE = (1 - pi) * (ATT - ATU)
    print('HTE', HTE)

    SDO = (
        np.mean(df.loc[df['assignment'] == 1, 'realized_outcome']) - 
        np.mean(df.loc[df['assignment'] == 0, 'realized_outcome'])
    )
    print('SDO:', SDO)

    decomposition = ATE + sb + (1 - pi) * (ATT - ATU)
    print('decomposition:', decomposition)

    Y = df['y0']
    X = df['assignment']
#     X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    print(results.summary())

In [67]:
assignment = 'perfect'
do_calculations(df, assignment)

ATE: 0.6002818365312486
ATT: 4.714841028208539
ATU: -4.273033743361979
{'E[y0 | D = 1]': 7.3521186175133755, 'E[y0 | D = 0]': 11.851908950479885}
selection bias: -4.499790332966509
pi: 0.54221
HTE 4.114559191677268
SDO: 0.21505069524207343
decomposition: 0.21505069524200682
                                 OLS Regression Results                                
Dep. Variable:                     y0   R-squared (uncentered):                   0.281
Model:                            OLS   Adj. R-squared (uncentered):              0.281
Method:                 Least Squares   F-statistic:                          3.906e+04
Date:                Sat, 04 Feb 2023   Prob (F-statistic):                        0.00
Time:                        12:12:20   Log-Likelihood:                     -3.5780e+05
No. Observations:              100000   AIC:                                  7.156e+05
Df Residuals:                   99999   BIC:                                  7.156e+05
Df Model:            

In [68]:
assignment = 'bad'
do_calculations(df, assignment)

ATE: 0.6002818365312486
ATT: 0.5648824521480714
ATU: 0.6356826369181084
{'E[y0 | D = 1]': 9.438603022224877, 'E[y0 | D = 0]': 9.385551184822628}
selection bias: 0.053051837402248836
pi: 0.50001
HTE -0.035399384383170814
SDO: 0.6179342895503357
decomposition: 0.6179342895503266
                                 OLS Regression Results                                
Dep. Variable:                     y0   R-squared (uncentered):                   0.427
Model:                            OLS   Adj. R-squared (uncentered):              0.427
Method:                 Least Squares   F-statistic:                          7.448e+04
Date:                Sat, 04 Feb 2023   Prob (F-statistic):                        0.00
Time:                        12:12:28   Log-Likelihood:                     -3.4645e+05
No. Observations:              100000   AIC:                                  6.929e+05
Df Residuals:                   99999   BIC:                                  6.929e+05
Df Model:         

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     y0   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                 4.633e+04
Date:                Sat, 04 Feb 2023   Prob (F-statistic):               0.00
Time:                        12:10:21   Log-Likelihood:            -2.6083e+05
No. Observations:              100000   AIC:                         5.217e+05
Df Residuals:                   99998   BIC:                         5.217e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.8465      0.015    769.951      0.000      11.816      11.877
assignment    -4.4897      0.021   -215.246      0.000      -4.531      -4.449
==============================================================================
Omnibus:                      104.429   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               97.077
Skew:                           0.048   Prob(JB):                     8.32e-22
Kurtosis:                       2.882   Cond. No.                         2.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""